### 슈팅 데이터 불러오기

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from src.plot_utils import get_pitch_layout

In [ ]:
shots = pd.read_pickle('data/shots.pkl')
shots

### 단일 경기 xG 시각화

##### (1) 슈팅 레이블 생성

In [ ]:
match_id = 2057987

match_df = pd.read_csv('data/refined_events/World_Cup/matches.csv', index_col=0, header=0, encoding='utf-8-sig')
team1_name = match_df.at[match_id, 'team1_name']
team2_name = match_df.at[match_id, 'team2_name']

match_shots = shots[shots['match_id'] == match_id]
match_shots['display_name'] = match_shots.apply(
    lambda x: f"{x['player_name']}, " +
    f"{x['period']} {int(x['time'] // 60):02d}:{int(x['time'] % 60):02d}, " +
    f"xG: {round(x['xg'], 3)}", axis=1
)

team2_x = match_shots.loc[match_shots['team_name'] == team2_name, 'x']
team1_y = match_shots.loc[match_shots['team_name'] == team1_name, 'y']
team2_y = match_shots.loc[match_shots['team_name'] == team2_name, 'y']
match_shots.loc[match_shots['team_name'] == team2_name, 'x'] = 104 - team2_x
match_shots.loc[match_shots['team_name'] == team1_name, 'y'] = 34 - team1_y
match_shots.loc[match_shots['team_name'] == team2_name, 'y'] = 34 + team2_y

match_shots.head()

##### (2) 슈팅 위치 및 xG 시각화

In [ ]:
match_shots_failed = match_shots[match_shots['tags'].apply(lambda x: 'Goal' not in x)]

team1_shots = match_shots[match_shots['team_name'] == team1_name]
team2_shots = match_shots[match_shots['team_name'] == team2_name]
team1_goals = team1_shots[team1_shots['tags'].apply(lambda x: 'Goal' in x)]
team2_goals = team2_shots[team2_shots['tags'].apply(lambda x: 'Goal' in x)]

team1_goal_trace = go.Scatter(
    x=team1_goals['x'],
    y=team1_goals['y'],
    name=team1_name,
    text=team1_goals['display_name'],
    mode='markers',
    marker=dict(
        color='red', size=np.sqrt(team1_goals['xg']) * 50, 
        symbol=team1_goals['freekick'].apply(lambda x: 'square' if x == 1 else 'circle')
    )
)

team2_goal_trace = go.Scatter(
    x=team2_goals['x'],
    y=team2_goals['y'],
    name=team2_name,
    text=team2_goals['display_name'],
    mode='markers',
    marker=dict(
        color='blue', size=np.sqrt(team2_goals['xg']) * 50, 
        symbol=team2_goals['freekick'].apply(lambda x: 'square' if x == 1 else 'circle')
    )
)

shot_trace = go.Scatter(
    x=match_shots_failed['x'],
    y=match_shots_failed['y'],
    name='Failed shot',
    text=match_shots_failed['display_name'],
    mode='markers',
    marker=dict(
        color='darkgrey', size=np.sqrt(match_shots_failed['xg']) * 50, 
        symbol=match_shots_failed['freekick'].apply(lambda x: 'square' if x == 1 else 'circle')
    )
)

team1_xg = team1_shots['xg'].sum().round(2)
team2_xg = team2_shots['xg'].sum().round(2)
title = f"{team1_name} - {team2_name} (xG: {team1_xg} - {team2_xg})"
fig = go.Figure(data=[shot_trace, team1_goal_trace, team2_goal_trace], layout=get_pitch_layout(title))
fig.show()

### 선수별 시즌 전체 xG 및 부가 지표 집계

##### (1) 선수별 슈팅/유효슈팅/득점 횟수 집계

In [ ]:
shots = shots[shots['competition_name'] == 'England']
shots_on_target = shots[shots['tags'].apply(lambda x: 'Accurate' in x)]
goals = shots[shots['tags'].apply(lambda x: 'Goal' in x)]

player_shot_counts = shots.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
player_sot_counts = shots_on_target.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
player_goal_counts = goals.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()

player_stats = pd.concat([player_shot_counts, player_sot_counts, player_goal_counts], axis=1).fillna(0).astype(int)
player_stats.columns = ['Shots', 'SoT', 'Goals']
player_stats

In [ ]:
player_stats.sort_values('Goals', ascending=False)[:20].reset_index()

##### (2) 선수별 xG 집계 및 정렬

In [ ]:
player_stats['xG'] = shots.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['xg'].sum()
player_stats.sort_values('xG', ascending=False)[:20].reset_index()

##### (3) 선수별 득점-xG 차이(dG) 집계 및 정렬

In [ ]:
player_stats['dG'] = player_stats['Goals'] - player_stats['xG']
player_stats.sort_values('dG', ascending=False)[:20].reset_index()

##### (4) 선수별 득점당 평균 xG 집계 및 정렬

In [ ]:
player_stats['xG for goals'] = goals.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['xg'].sum()
player_stats['xG per goal'] = player_stats['xG for goals'] / player_stats['Goals']
player_stats[player_stats['Goals'] >= 5].sort_values('xG per goal')[:20].reset_index()

### 선수별 시즌 전체 xG 시각화

##### (1) 슈팅 레이블 생성

In [ ]:
match_df = pd.read_csv('data/refined_events/England/matches.csv', header=0, encoding='utf-8-sig')
shots = pd.merge(shots, match_df[['match_id', 'gameweek', 'team1_name', 'team2_name']])
shots['display_name'] = shots.apply(
    lambda x: f"[{x['gameweek']}R] {x['team1_name']} vs {x['team2_name']}, " +
    f"{x['period']} {int(x['time'] // 60):02d}:{int(x['time'] % 60):02d}, xG: {round(x['xg'], 3)}", axis=1
)
shots

##### (2) 슈팅 위치 및 xG 시각화

In [ ]:
player_name = 'Son Heung-Min'
# player_name = 'Mohamed Salah'
# player_name = 'K. De Bruyne'

player_shots = shots[shots['player_name'] == player_name]
player_goals = player_shots[player_shots['tags'].apply(lambda x: 'Goal' in x)]
player_shots_failed = player_shots[player_shots['tags'].apply(lambda x: 'Goal' not in x)]

goal_trace = go.Scatter(
    x=104 - player_goals['x'],
    y=34 + player_goals['y'],
    name='Goal',
    text=player_goals['display_name'],
    mode='markers',
    marker=dict(
        color='red', size=np.sqrt(player_goals['xg']) * 50, 
        symbol=player_goals['freekick'].apply(lambda x: 'square' if x == 1 else 'circle')
    )
)

shot_trace = go.Scatter(
    x=104 - player_shots_failed['x'],
    y=34 + player_shots_failed['y'],
    name='Failed shot',
    text=player_shots_failed['display_name'],
    mode='markers',
    marker=dict(
        color='darkgrey', size=np.sqrt(player_shots_failed['xg']) * 50, 
        symbol=player_shots_failed['freekick'].apply(lambda x: 'square' if x == 1 else 'circle')
    )
)

title = f"{player_name} - Goals: {player_shots['goal'].sum()}, xG: {round(player_shots['xg'].sum(), 3)} "
fig = go.Figure(data=[shot_trace, goal_trace], layout=get_pitch_layout(title))
fig.show()